### Running a single-machine cluster
When run on a single machine, this is equivalent to the dask 'processes' scheduler, with the added addition of a handy status dashboard. It's accessible by web browser via the link below. (The local cluster server can be set up and run outside the notebook for persistence.)

In [1]:
from dask import distributed

CLUSTER_SETTINGS = dict(
    # limiting this can help to avoid system hangs caused by google drivefs
    n_workers=8,
    threads_per_worker=1,
    # host=f"tcp://127.0.0.1:8786",
    dashboard_address=f"127.0.0.1:8787",
)

# cluster = distributed.LocalCluster(**CLUSTER_SETTINGS)
if 'scheduler' not in dir():
    cluster = distributed.LocalCluster(**CLUSTER_SETTINGS)
    scheduler = distributed.Client(cluster.scheduler_address)

cluster

2023-03-14 12:54:41,257 - bokeh.server.util - WARNING - Host wildcard '*' will allow connections originating from multiple (or possibly all) hostnames or IPs. Use non-wildcard values to restrict access explicitly


LocalCluster(a1bb8312, 'tcp://127.0.0.1:15938', workers=8, threads=8, memory=31.60 GiB)

### Creating dask dataframe objects
The following assumes that you've downloaded the specified zip archive at the specified path.

In [2]:
from dask_ops import log_to_json
import __init__ as sea_ingest
from labbench import stopwatch
import pandas as pd
import typing

def capture_summary(partition_data: typing.Dict[str, pd.DataFrame]):
    """ this could be expanded into a function that does more, but for now, just make a capture summary """

    # partition_data is the result of running zipfile.read_seamf_zipfile on a subset of files in the zip archive.
    # it takes the form of a dictionary of pandas.DataFrame objects

    partition_data['capture_summary'] = pd.DataFrame.from_dict({
        'median_rms_pfp': 
            sea_ingest.trace(partition_data, 'pfp', capture_statistic='mean', detector='rms').median(axis=1),
        'max_max_pfp': 
            sea_ingest.trace(partition_data, 'pfp', capture_statistic='max', detector='peak').max(axis=1),
        'median_mean_power': 
            sea_ingest.trace(partition_data, 'psd', capture_statistic='mean').median(axis=1),
        'max_max_power': 
            sea_ingest.trace(partition_data, 'psd', capture_statistic='mean').max(axis=1),
        'noise_figure': 
            sea_ingest.trace(partition_data, 'channel_metadata')['cal_noise_figure_dB'].astype('float16'),
        'gain': 
            sea_ingest.trace(partition_data, 'channel_metadata')['cal_gain_dB'].astype('float16'),
        # TODO: add 'temperature' from sweep_metadata, will probably need a restructure of that table
    })

    return partition_data

data_path = 'data/NIT-2022-12-13.zip'

# overwrite
log_to_json('data.log')

with stopwatch('setup'):
    # scan the zipfiles to map out the file contents. the resulting is a dictionary of dask dataframes.
    # this is fast, because it hasn't loaded much data yet; other operations trigger "compute" that can
    # take a while to scrape the data out of the zip archive
    ddfs = sea_ingest.read_seamf_zipfile_as_ddf(
        data_path,
        partition_func=capture_summary,
        partition_size=100,
    )

C:\Users\dkuester\AppData\Local\pdm\pdm\Cache\packages\eliot-1.14.0-py2.py3-none-any\lib\eliot\json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if isinstance(o, (numpy.bool, numpy.bool_)):
C:\Users\dkuester\AppData\Local\pdm\pdm\Cache\packages\dask-2023.3.1-py3-none-any\lib\dask\base.py:1369: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(
 INFO   2023-03-14 12:54:51.216 • labbench: setup 2.657 s elapsed


In [3]:
with stopwatch('compute'):
    # head forces a compute operation
    df = ddfs['capture_summary'].head(10)
df

 INFO   2023-03-14 12:54:55.089 • labbench: compute 3.801 s elapsed


,frequency,median_rms_pfp,max_max_pfp,median_mean_power,max_max_power,noise_figure,gain
datetime,,,,,,,
2022-10-26 20:47:37.094,3.555000e+09,-86.1875,-64.81250,-155.375,-152.000,4.332031,31.437500
2022-10-26 20:47:41.699,3.565000e+09,-88.5000,-66.06250,-157.250,-154.125,4.265625,31.500000
2022-10-26 20:47:46.374,3.575000e+09,-86.2500,-64.00000,-155.750,-152.875,4.257812,31.312500
2022-10-26 20:47:51.554,3.585000e+09,-88.0625,-65.31250,-156.000,-153.875,4.031250,31.593750
2022-10-26 20:47:56.944,3.595000e+09,-82.7500,-60.12500,-150.125,-146.375,4.003906,31.718750
2022-10-26 20:48:01.623,3.605000e+09,-83.5625,-60.87500,-150.750,-147.875,3.990234,31.671875
2022-10-26 20:48:07.579,3.615000e+09,-83.8750,-64.06250,-153.250,-150.000,4.089844,31.531250
2022-10-26 20:48:12.256,3.625000e+09,-86.8750,-64.12500,-155.500,-152.625,4.054688,31.531250
2022-10-26 20:48:16.953,3.635000e+09,-79.7500,-59.28125,-151.375,-145.625,3.998047,31.687500


#### on-demand access to a subset of the data
The dask .loc accessor allows faster access to a specified subset of the data

In [4]:
with stopwatch('compute'):
    df = (
        ddfs['capture_summary']
        .loc['2022-11-21 20:50:00':'2022-11-21 20:51:00']
        .compute(scheduler=scheduler)
    )

df

 INFO   2023-03-14 12:55:04.463 • labbench: compute 9.289 s elapsed


,frequency,median_rms_pfp,max_max_pfp,median_mean_power,max_max_power,noise_figure,gain
datetime,,,,,,,
2022-11-21 20:50:03.159,3.555000e+09,-83.0000,-62.15625,-151.625,-148.875,4.109375,31.687500
2022-11-21 20:50:07.794,3.565000e+09,-87.1875,-63.03125,-152.750,-149.250,4.023438,31.765625
2022-11-21 20:50:12.536,3.575000e+09,-87.1250,-65.25000,-154.875,-151.750,4.035156,31.562500
2022-11-21 20:50:17.761,3.585000e+09,-89.4375,-66.75000,-156.125,-152.875,3.800781,31.843750
2022-11-21 20:50:23.217,3.595000e+09,-79.1250,-57.59375,-148.250,-144.250,3.769531,31.984375
2022-11-21 20:50:27.918,3.605000e+09,-79.2500,-57.53125,-147.375,-143.875,3.744141,31.921875
2022-11-21 20:50:34.832,3.615000e+09,-80.5000,-61.75000,-150.375,-147.625,3.853516,31.796875
2022-11-21 20:50:39.527,3.625000e+09,-82.4375,-62.09375,-151.250,-149.125,3.832031,31.765625
2022-11-21 20:50:44.269,3.635000e+09,-79.8750,-58.71875,-149.000,-145.625,3.753906,31.953125


#### Bulk data
Dask dataframes support data save operations split output files by partition. This means that the time span of each output file can be adjusted with 'repartition'. In this example, a file save function is applied to each partition.

In [5]:
import dask
from pathlib import Path

def write_feather(df: pd.DataFrame, dirpath):
    """ example for feather, but could substitute code for e.g. csv, database, etc. instead """
    path = Path(dirpath)/df.index[0].strftime('%Y-%m-%d.feather')

    # feather like most formats requires string column names
    df.columns = df.columns.astype('str')
    df.reset_index().to_feather(path, compression='zstd')
    return str(path)

# collecting the write operations into a single options
# allows dask to optimize the execution so that the zip archives
# only need to be read once.
#
# the choice of dictionary means that the returned list of
# files written will be returned as a dictionary with the same keys.
ddf = ddfs['capture_summary']

save_ops = dict(
    capture_summary=(
        ddfs['capture_summary']
        .repartition(freq='1MS')
        .map_partitions(write_feather, 'data/capture_summary')
    ),

    pfp=(
        ddfs['pfp']
        .repartition(freq='1W')
        .map_partitions(write_feather, 'data/pfp')
    ),
)

with stopwatch('compute'):
    files_saved = dask.compute(save_ops, scheduler=scheduler)

    print('wrote the following files:')
    print(files_saved)

 INFO   2023-03-14 13:00:17.473 • labbench: compute 312.762 s elapsed


wrote the following files:
({'capture_summary': 0    data\capture_summary\2022-10-26.feather
1    data\capture_summary\2022-11-01.feather
2    data\capture_summary\2022-12-01.feather
dtype: object, 'pfp': 0    data\pfp\2022-10-26.feather
1    data\pfp\2022-10-30.feather
2    data\pfp\2022-11-06.feather
3    data\pfp\2022-11-13.feather
4    data\pfp\2022-11-20.feather
5    data\pfp\2022-11-27.feather
6    data\pfp\2022-12-04.feather
dtype: object},)
